<a href="https://colab.research.google.com/github/RodolfoFerro/unet-workshop/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementación de U-Net con TensorFlow

> Contenido creado por **Rodolfo Ferro** ([Future Lab](https://futurelab.mx/), 2021). <br>
> Contacto: [@rodo_ferro](https://www.instagram.com/rodo_ferro/) (en Instagram) & [@rodo_ferro](https://twitter.com/rodo_ferro) (en Twitter)

En este cuaderno podrás encontrar un código base que implementa el modelo de UNet para realizar un entrenamiento e inferencia de imágenes médicas.

## Clonamos el repositorio

Comenzaremos clonando el repositorio y asignando a la carpeta como la raíz.

In [ ]:
!git clone https://github.com/RodolfoFerro/unet-workshop.git
%cd unet-workshop
!ls

La estructura del código fuente es como sigue:
- `model.py` - Contiene la implementación del U-Net.
- `data.py` - Contiene funciones de utilería para carga de datos.
- `image.py` - Coniene funciones de utilería para cargar imágenes y mostrar los resultados de las inferencias.
- `main.py` - Contiene una sencilla implementación de este cuaderno en un script de Python para entrenar el modelo.

------

## Importamos el modelo

A continuación procedemos a importar algunas bibliotecas y el código base del modelo.

Haremos uso de alunas funciones que permiten cargar datos que encuentras en el folder `data`.

Comenzaremos importanto los módulos apra poder utilizar algunas funciones.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

from model import unet
from data import train_generator
from data import test_generator
from data import save_results

Procedemos a crear un diccionario de configuración para cargar datos.

In [ ]:
data_gen_args = dict(
    rotation_range=0.2,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = train_generator(
    2, 'data/membrane/train',
    'image', 'label',
    data_gen_args,
    save_to_dir=None
)

Creamos una instancia del modelo y entrenamos con los datos.

In [ ]:
model = unet()
model_checkpoint = ModelCheckpoint(
    'unet_membrane.hdf5',
    monitor='loss',
    verbose=1,
    save_best_only=True
)

model.fit(
    train_gen,
    steps_per_epoch=300,
    epochs=5,
    callbacks=[model_checkpoint]
)

**¡Felicidades! Has entrenado exitosamente un U-Net con algunas imágenes médicas.**

Una vez entrenado el modelo, podemos realizar pruebas decon el conjunto de pruebas que se encuentra en la misma carpeta de datos.

In [ ]:
test_gen = test_generator('data/membrane/test')
results = model.predict(test_gen, 30, verbose=True)
save_results('data/results', results)

------

## Resultados gráficos

El código base provee algunas funciones para cargar, inferir y crear máscaras de los resultados al trabajar sobre algunas imágenes.

Procedemos a importar las funciones del módulo de imágenes.

In [ ]:
from image import load_test_image
from image import inference_over_image
from image import create_mask
from image import overlay_mask

Cargamos una imagen del directorio de prueba, especificando con un número entero el índice de alguna de las 30 imágenes (`[0, 29]`).

In [ ]:
img = load_test_image(0)

Usamos el modelo previamente entrenado para inferir sobre la imagen previamente cargada.

In [ ]:
out = inference_over_image(model, img)

Creamos una máscara a partir de la inferencia.

In [ ]:
mask = create_mask(out)

Sobreponemos la máscara en la imágen original para validar el resulatdo.

In [ ]:
res = overlay_mask(img, mask)

**¡Felicidades! Has utilizado exitosamente tu modelo entrenado sobre algunas imágenes médicas.**